In [ ]:
import { display } from "tslab";
import { readFileSync } from "fs";

const css = readFileSync("../style.css", "utf8");
display.html(`<style>${css}</style>`);

# Minimizing a Finite State Machine

In this notebook, we show how a **DFA** (Deterministic Finite Automaton) can be minimized by identifying states that are equivalent.

We will strictly adhere to the type definitions established in the previous notebooks.

In [ ]:
import { RecursiveSet, Tuple } from "recursive-set";
import { key } from "./05-DFA-2-RegExp";

## Type Definitions

We use the existing types from our DFA implementation.

* `State`: Atomic state identifier (string or number).
* `DFAState`: A state in our DFA (which itself is a set of NFA states).
* `TransRelDet`: The deterministic transition function $\delta$.
* `DFA`: The object structure.

We introduce:
* `Pair`: A tuple of two states $(p, q)$.
* `MinState`: A state in the minimized DFA, which is a set of equivalent original states (an equivalence class).

In [ ]:
// === Existing Types ===
type State = string | number;
type Char = string;

// A state in our current DFA (potentially formed by subset construction)
type DFAState = RecursiveSet<State>; 

type TransRelDet = Map<string, DFAState>;

type DFA = {
  Q: RecursiveSet<DFAState>;
  Sigma: RecursiveSet<Char>;
  delta: TransRelDet;
  q0: DFAState;
  A: RecursiveSet<DFAState>;
};

// === New Types for Minimization ===

// A pair of states (p, q)
type Pair = Tuple<[DFAState, DFAState]>;

// A state in the Minimized DFA is a set of original DFAStates (Equivalence Class)
type MinState = RecursiveSet<DFAState>;

type MinTransRel = Map<string, MinState>;

type MinDFA = {
    Q: RecursiveSet<MinState>;
    Sigma: RecursiveSet<Char>;
    delta: MinTransRel;
    q0: MinState;
    A: RecursiveSet<MinState>;
}

### Helper Functions

**Arbitrary Element:**
The function `arb(M)` returns an *arbitrary* element from a set `M`.
Since `RecursiveSet` implements the iterator protocol and is internally sorted, this function deterministically returns the "smallest" element.

**Cartesian Product:**
The function `cartProd(A, B)` computes the Cartesian product $A \times B$. We leverage the built-in `cartesianProduct` method.

In [ ]:
function arb<T>(M: RecursiveSet<T>): T {
    for (const x of M) {
        return x;
    }
    throw new Error("Error: arb called with empty set!");
}

function cartProd<S, T>(A: RecursiveSet<S>, B: RecursiveSet<T>): RecursiveSet<Tuple<[S, T]>> {
    return A.cartesianProduct(B);
}

### Separation Logic

The function `separate` takes four arguments:
- `Pairs`: A set of pairs known to be separable.
- `States`: The set of all states $Q$.
- `Sigma`: The alphabet $\Sigma$.
- `delta`: The transition map.

It returns a **new** set of separable pairs. Two states $q_1, q_2$ become separable if there exists a character $c$ such that their targets $p_1 = \delta(q_1, c)$ and $p_2 = \delta(q_2, c)$ are already known to be separable.

In [ ]:
function separate(
    Pairs: RecursiveSet<Pair>, 
    States: RecursiveSet<DFAState>, 
    Sigma: RecursiveSet<Char>, 
    delta: TransRelDet
): RecursiveSet<Pair> {
    
    const result = new RecursiveSet<Pair>();
    
    for (const q1 of States) {
        for (const q2 of States) {
            for (const c of Sigma) {
                // Get targets
                const p1 = delta.get(key(q1, c));
                const p2 = delta.get(key(q2, c));
                
                // If transitions exist
                if (p1 && p2) {
                    // Check if the pair of targets (p1, p2) is already in the Separable set
                    // We must construct the tuple to check existence in RecursiveSet
                    const targetPair = new Tuple(p1, p2);
                    
                    if (Pairs.has(targetPair)) {
                        result.add(new Tuple(q1, q2));
                    }
                }
            }
        }
    }
    
    return result;
}

### Equivalence Class Lookup

Given a state `p` and a partition of the states (a set of sets), this function returns the specific set (equivalence class) that contains `p`.

In [ ]:
function findEquivalenceClass(
    p: DFAState, 
    Partition: RecursiveSet<MinState>
): MinState {
    // Find the set C in Partition such that p is in C
    for (const C of Partition) {
        if (C.has(p)) {
            return C;
        }
    }
    throw new Error(`State ${p} not found in partition`);
}

### Computing All Separable Pairs

The function `allSeparable` computes the set of all pairs $(p, q)$ that are **distinguishable**.

It uses a **fixed-point iteration**:
1.  **Base Case:** Start with pairs where one state is accepting and the other is not: $(Q \setminus A) \times A \cup A \times (Q \setminus A)$.
2.  **Iteration:** Repeatedly apply `separate` to find pairs that lead to already separated pairs. Stop when no new pairs are added.

In [ ]:
function allSeparable(
    Q: RecursiveSet<DFAState>, 
    A: RecursiveSet<DFAState>, 
    Sigma: RecursiveSet<Char>, 
    delta: TransRelDet
): RecursiveSet<Pair> {
    
    // Q without A (Non-accepting states)
    const NonAccepting = new RecursiveSet<DFAState>();
    for(const q of Q) {
        if(!A.has(q)) NonAccepting.add(q);
    }

    // 1. Initial Separation: (NonAccept x Accept) U (Accept x NonAccept)
    const set1 = cartProd(NonAccepting, A);
    const set2 = cartProd(A, NonAccepting);
    
    // Combine them
    const Separable = new RecursiveSet<Pair>();
    for(const p of set1) Separable.add(p);
    for(const p of set2) Separable.add(p);

    // 2. Fixed Point Loop
    while (true) {
        const NewPairs = separate(Separable, Q, Sigma, delta);
        
        // Check if NewPairs is a subset of Separable
        let isSubset = true;
        for (const np of NewPairs) {
            if (!Separable.has(np)) {
                isSubset = false;
                break;
            }
        }
        
        if (isSubset) {
            return Separable;
        }
        
        // Union: Separable |= NewPairs
        for (const np of NewPairs) {
            Separable.add(np);
        }
    }
}

### Reachability

The function `reachable` filters out states that cannot be reached from the start state $q_0$. Unreachable states are irrelevant for the language accepted by the DFA. It also uses a fixed-point iteration.

In [ ]:
function reachable(
    q0: DFAState, 
    Sigma: RecursiveSet<Char>, 
    delta: TransRelDet
): RecursiveSet<DFAState> {
    
    const Result = new RecursiveSet<DFAState>(q0);
    
    while (true) {
        const NewStates = new RecursiveSet<DFAState>();
        
        // Find all states reachable in one step from current set
        for (const p of Result) {
            for (const c of Sigma) {
                const target = delta.get(key(p, c));
                if (target) {
                    NewStates.add(target);
                }
            }
        }
        
        // Check for subset (Fixpoint check)
        let isSubset = true;
        for (const ns of NewStates) {
            if (!Result.has(ns)) {
                isSubset = false;
                break;
            }
        }
        
        if (isSubset) {
            return Result;
        }
        
        // Result |= NewStates
        for (const ns of NewStates) {
            Result.add(ns);
        }
    }
}

### The Minimization Algorithm

The `minimize` function puts it all together:
1.  Remove unreachable states.
2.  Calculate all separable pairs.
3.  Identify **equivalent pairs** (Total Pairs minus Separable Pairs).
4.  Group equivalent states into **Equivalence Classes**.
5.  Construct the new Minimized DFA where states are the equivalence classes.

**Note:** We define a local helper `minKey` because the states of the minimized DFA are sets of sets (`MinState`), so the original `key` function (which expects sets of primitives) is not type-compatible.

In [ ]:
// Local helper to generate keys for the minimized states (Sets of Sets)
function minKey(q: MinState, c: Char): string {
    return `${q.toString()},${c}`;
}

function minimize(F: DFA): MinDFA {
    let { Q, Sigma, delta, q0, A } = F;
    
    // 1. Filter Reachable States
    Q = reachable(q0, Sigma, delta);
    
    // 2. Calculate Separable Pairs
    const Separable = allSeparable(Q, A, Sigma, delta);
    
    // 3. Calculate Equivalent Pairs (Total - Separable)
    const EquivClasses = new RecursiveSet<MinState>();
    
    for (const q of Q) {
        const equivalenceClass = new RecursiveSet<DFAState>();
        for (const p of Q) {
            const pairToCheck = new Tuple(p, q);
            if (!Separable.has(pairToCheck)) {
                equivalenceClass.add(p);
            }
        }
        EquivClasses.add(equivalenceClass);
    }
    
    // 4. Construct New Start State
    let newQ0: MinState | undefined;
    for (const M of EquivClasses) {
        if (M.has(q0)) {
            newQ0 = M;
            break;
        }
    }
    if (!newQ0) throw new Error("Start state vanished!");

    // 5. Construct New Accepting States
    const newAccept = new RecursiveSet<MinState>();
    for (const M of EquivClasses) {
        const representative = arb(M);
        if (A.has(representative)) {
            newAccept.add(M);
        }
    }
    
    // 6. Construct New Delta
    const newDelta: MinTransRel = new Map();
    
    for (const q of Q) {
        for (const c of Sigma) {
            const p = delta.get(key(q, c));
            
            const classOfQ = findEquivalenceClass(q, EquivClasses);
            
            if (p) {
                const classOfP = findEquivalenceClass(p, EquivClasses);
                // Fix: Use minKey here instead of imported key
                newDelta.set(minKey(classOfQ, c), classOfP);
            }
        }
    }
    
    return {
        Q: EquivClasses,
        Sigma: Sigma,
        delta: newDelta,
        q0: newQ0,
        A: newAccept
    };
}